In [ ]:
import numpy as np
import cv2
import os
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt


In [ ]:
def load_images_from_folder(folder):
    images = []
    labels = []
    for label, subfolder in enumerate(os.listdir(folder)):
        subfolder_path = os.path.join(folder, subfolder)
        for filename in os.listdir(subfolder_path):
            img = cv2.imread(os.path.join(subfolder_path, filename), cv2.IMREAD_GRAYSCALE)
            if img is not None:
                img = cv2.resize(img, (100, 100))  # Resize images to 100x100
                images.append(img.flatten())  # Convert the image to a 1D vector
                labels.append(label)
    return np.array(images), np.array(labels)

# Load the dataset
X, y = load_images_from_folder('faces')  # Replace with your dataset path


In [11]:
def pca(X, k):
    # Step 1: Compute the mean of the images
    mean_face = np.mean(X, axis=0)
    
    # Step 2: Subtract the mean face
    X_centered = X - mean_face
    
    # Step 3: Compute the covariance matrix
    covariance_matrix = np.cov(X_centered.T)
    
    # Step 4: Perform eigen decomposition
    eigenvalues, eigenvectors = np.linalg.eig(covariance_matrix)
    
    # Step 5: Sort eigenvectors based on eigenvalues in descending order
    idx = np.argsort(eigenvalues)[::-1]
    eigenvectors = eigenvectors[:, idx][:, :k]  # Select the top k eigenvectors
    
    # Step 6: Project the centered images onto the eigenvectors
    X_pca = np.dot(X_centered, eigenvectors)
    
    return X_pca, mean_face, eigenvectors

# Choose the number of principal components (k)
k = 50
X_pca, mean_face, eigenvectors = pca(X, k)


In [12]:
X_pca_real = np.real(X_pca)
# Split the dataset into training and testing sets (60% train, 40% test)
X_train, X_test, y_train, y_test = train_test_split(X_pca_real, y, test_size=0.4, random_state=42)

# Train an Artificial Neural Network (ANN) using MLPClassifier
clf = MLPClassifier(hidden_layer_sizes=(100,), max_iter=500, alpha=0.001, solver='adam', random_state=42)
clf.fit(X_train, y_train)

# Test the classifier
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')


Accuracy: 46.67%


In [13]:
def test_face(test_img, mean_face, eigenvectors):
    # Preprocess the test image
    img = cv2.resize(test_img, (100, 100)).flatten()
    
    # Subtract the mean face
    img_centered = img - mean_face
    
    # Project the test image onto the eigenvectors
    img_projected = np.dot(img_centered, eigenvectors)
    img_projected_real = np.real(img_projected)
    
    # Use the trained classifier to predict the label
    return clf.predict([img_projected_real])

# Load a test image
test_img = cv2.imread('test.jpg', cv2.IMREAD_GRAYSCALE)  # Replace with your test image path

# Predict the class for the test image
prediction = test_face(test_img, mean_face, eigenvectors)
print(f'Predicted label: {prediction}')


Predicted label: [4]


In [17]:
def recognize_imposter(test_img, mean_face, eigenvectors):
    # Preprocess the test image (as done for other images)
    img = cv2.resize(test_img, (100, 100)).flatten()
    img_centered = img - mean_face
    img_projected = np.dot(img_centered, eigenvectors)
    img_projected_real = np.real(img_projected)
    # Predict the class
    prediction = clf.predict([img_projected_real])
    
    # If the predicted class is unknown, recognize as an imposter
    # (Assuming you know the list of valid classes)
    valid_classes = np.unique(y)  # All known classes
    if prediction not in valid_classes:
        return "Imposter"
    else:
        return f'Predicted label: {prediction}'

# Test with a possible imposter image
imposter_img = cv2.imread('imposter.jpg', cv2.IMREAD_GRAYSCALE)  # Replace with imposter image path
imposter_result = recognize_imposter(imposter_img, mean_face, eigenvectors)
print(imposter_result)


Predicted label: [8]


In [18]:
def plot_accuracy_vs_k(X, y, k_values):
    accuracies = []
    
    for k in k_values:
        X_pca, _, _ = pca(X, k)
        X_pca_real = np.real(X_pca)
        X_train, X_test, y_train, y_test = train_test_split(X_pca_real, y, test_size=0.4, random_state=42)
        clf = MLPClassifier(hidden_layer_sizes=(100,), max_iter=500, alpha=0.001, solver='adam', random_state=42)
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        accuracies.append(accuracy_score(y_test, y_pred))
    
    plt.plot(k_values, accuracies)
    plt.xlabel('k value')
    plt.ylabel('Accuracy')
    plt.title('Accuracy vs k Value')
    plt.show()

# Define a range of k values to test
k_values = [10, 20, 30, 40, 50, 60, 70]
plot_accuracy_vs_k(X, y, k_values)